In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install evalml


In [3]:
df=pd.read_excel('final_feat.xlsx')

In [28]:
df1=df[['Completed Jobs','TMAX','Avg Sales','tech_count','Income_y']]

In [29]:
df1=df1.drop_duplicates(ignore_index=True)

In [45]:

df1['f1']=np.sqrt(df1['Completed Jobs']**3)*np.abs(df1['Avg Sales'])
df1['f2']=np.abs(np.sqrt(df1['Completed Jobs']) - np.sqrt(df1['tech_count']))
df1['f3']=np.log(np.sqrt(df1['Completed Jobs'])*df1['TMAX']**3)

In [53]:
df1.columns

Index(['Completed Jobs', 'TMAX', 'Avg Sales', 'tech_count', 'Income_y', 'f1',
       'f2', 'f3'],
      dtype='object')

In [54]:
df2=df1[['Income_y', 'f1',
       'f2', 'f3']]

In [55]:
train=df2[0:300]
test=df2[300:]

In [56]:
y=train['Income_y']
X=train.drop(['Income_y'],axis=1)

In [57]:
!pip install fsspec

In [16]:
!pip install dask distributed --upgrade

Requirement already up-to-date: dask in /usr/local/lib/python3.7/dist-packages (2021.4.1)
Requirement already up-to-date: distributed in /usr/local/lib/python3.7/dist-packages (2021.4.1)


In [17]:
import evalml


In [58]:
X

,f1,f2,f3
0,1.556733e+06,0.131311,14.544687
1,1.203606e+06,2.450409,14.959942
2,1.873234e+05,2.833623,14.705986
3,1.243361e+04,0.550510,13.598199
4,1.135194e+06,0.798037,15.176646
...,...,...,...
295,1.446820e+06,3.710831,15.328614
296,2.203351e+06,2.723805,14.895628
297,1.580055e+06,4.609796,14.638705
298,5.043050e+05,2.947962,14.032067


In [59]:
X_train, X_test, y_train, y_test = evalml.preprocessing.split_data(X, y, problem_type='regression')


In [60]:

from evalml.automl import AutoMLSearch
automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='regression')
automl.search()

Using default limit of max_batches=1.

Generating pipelines to search over...

*****************************
* Beginning pipeline search *
*****************************

Optimizing for R2. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: random_forest, catboost, lightgbm, xgboost, extra_trees, decision_tree, linear_model



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mean Baseline Regression Pipeline
Mean Baseline Regression Pipeline:
	Starting cross validation
	Finished cross validation - mean R2: -0.017
	High coefficient of variation (cv >= 0.2) within cross validation scores.
	Mean Baseline Regression Pipeline may not perform as estimated on unseen data.

*****************************
* Evaluating Batch Number 1 *
*****************************

Decision Tree Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.509
[19:42:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:42:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:42:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2:

In [61]:
automl.rankings


,id,pipeline_name,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,5,Extra Trees Regressor w/ Imputer,0.661622,0.050655,0.627533,4039.221800,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,4,Linear Regressor w/ Imputer + Standard Scaler,0.660635,0.085277,0.618821,4033.346129,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,8,Elastic Net Regressor w/ Imputer + Standard Sc...,0.658905,0.065904,0.616914,4023.048173,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,3,Random Forest Regressor w/ Imputer,0.656945,0.041997,0.641447,4011.380280,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,6,LightGBM Regressor w/ Imputer,0.652779,0.031083,0.637179,3986.572395,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,2,XGBoost Regressor w/ Imputer,0.545340,0.012163,0.553978,3346.892899,False,{'Imputer': {'categorical_impute_strategy': 'm...
6,1,Decision Tree Regressor w/ Imputer,0.508509,0.072174,0.441946,3127.608461,False,{'Imputer': {'categorical_impute_strategy': 'm...
7,7,CatBoost Regressor w/ Imputer,0.236377,0.004594,0.241195,1507.361933,False,{'Imputer': {'categorical_impute_strategy': 'm...
8,0,Mean Baseline Regression Pipeline,-0.016796,0.014729,-0.000863,0.000000,True,{'Baseline Regressor': {'strategy': 'mean'}}


In [62]:
automl.best_pipeline


GeneratedPipeline(parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'Extra Trees Regressor':{'n_estimators': 100, 'max_features': 'auto', 'max_depth': 6, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_jobs': -1},})

In [63]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])



************************************
* Extra Trees Regressor w/ Imputer *
************************************

Problem Type: regression
Model Family: Extra Trees

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. Extra Trees Regressor
	 * n_estimators : 100
	 * max_features : auto
	 * max_depth : 6
	 * min_samples_split : 2
	 * min_weight_fraction_leaf : 0.0
	 * n_jobs : -1

Training
Training for regression problems.
Total training time (including CV): 0.8 seconds

Cross Validation
----------------
               R2  ExpVariance   MaxError  MedianAE            MSE       MAE  Root Mean Squared Error # Training # Validation
0           0.628        0.629 190227.404 22200.182 1812457537.029 29411.788                42572.967        160           80
1           0.720        0.723 106532.830 17084.288 1246609720.634 26327.764                35307.361        160    

In [64]:
y_=test['Income_y']
X_=test.drop(['Income_y'],axis=1)

In [65]:
best_pipeline=automl.best_pipeline


In [66]:
X_

,f1,f2,f3
300,1.868345e+06,4.290656,14.784169
301,1.827381e+06,3.635273,15.267597
302,1.592383e+06,3.321237,15.253261
303,2.051410e+06,2.496180,14.861992
304,2.096939e+06,4.687567,14.899957
305,4.726621e+05,3.157004,14.815144
306,3.260000e+02,1.828427,13.292450
307,8.963362e+05,3.154202,15.624170
308,1.443645e+06,3.099505,15.264951
309,8.360290e+05,2.275236,14.701497


In [67]:
best_pipeline.score(X_, y_, objectives=["mse","mae"])


OrderedDict([('MSE', 894093581.646812), ('MAE', 23485.941838602)])

In [69]:
np.sqrt(894093581.646812)

29901.397653735385